### 问题检测

主要实现了将非问题问答对剔除，并且将答案去除html格式（正则化）

In [2]:
import nltk
nltk.download('nps_chat')

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\wang\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


True

In [3]:
# 将语料库的多分类标签转化为二分类标签
def handle_y(list):
    for i in range(len(y)):
        if y[i] != 'whQuestion' and y[i] != 'ynQuestion':
            y[i] = 'noQuestion'
        if y[i] == 'whQuestion' or y[i] == 'ynQuestion':
            y[i] = 'Question'
    return y

In [4]:
# 进行修正
question_pattern = [ 'wonder which','do you', 'does it','does he',"don't you",'does she']
helping_verbs = ["is","am","can", "are", "does","should","would", "what", "who", "why", "how", "where", "when"]

def is_question(question):
    question = question.lower().strip()
    is_ques = False
    # 检查是否包括标志词
    for pattern in question_pattern:
        is_ques  = pattern in question
        if is_ques:
            break
    # 开头如果是这些词就说明是疑问句
    sentence_arr = question.split(".")
    for sentence in sentence_arr:
        if len(sentence.strip()):
            first_word = nltk.word_tokenize(sentence)[0]
            if first_word in helping_verbs:
                is_ques = True
                break
    return is_ques    

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import AdaBoostClassifier as ada
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False# 用来正常显示负号

In [6]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier   
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [7]:
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()
posts_text = [post.text for post in posts]
#划分训练集、测试集
train_text = posts_text[:int(len(posts_text)*0.95)]
test_text = posts_text[int(len(posts_text)*0.05):]

[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\wang\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


In [8]:
#提取TFIDF特征
vectorizer = TfidfVectorizer(ngram_range=(1,3), 
                             min_df=0.001, 
                             max_df=0.7, 
                             analyzer='word')

X_train = vectorizer.fit_transform(train_text)
X_test = vectorizer.transform(test_text)

In [30]:
vectorizer

TfidfVectorizer(max_df=0.7, min_df=0.001, ngram_range=(1, 3))

In [9]:
y = [post.get('class') for post in posts]


In [10]:
#处理y的标签，只分为两类，Question 和 noQuestion
y = handle_y(y)
y_train = y[:int(len(posts_text)*0.95)]
y_test = y[int(len(posts_text)*0.05):]

### 随机森林

In [11]:
rf = RandomForestClassifier(n_estimators = 500, random_state=0)
rf.fit(X_train, y_train)
y_rf_pred = rf.predict(X_test)

In [12]:
zql_rf = metrics.accuracy_score(y_rf_pred,y_test)
print('预测准确率', zql_rf)
# print(classification_report(y_test, y_rf_pred))

预测准确率 0.9826675963741408


### 逻辑回归

In [13]:
LR = LogisticRegression(C=0.01, class_weight=None,dual=False,fit_intercept=True,
                        intercept_scaling=1,l1_ratio=None, max_iter=100,
                       multi_class='auto',n_jobs=None,penalty='l2',random_state=None,
                        solver='newton-cg',tol = 0.001, verbose = 0,warm_start=False)
LR = LR.fit(X_train, y_train)

y_lr_pred = LR.predict(X_test)

In [14]:
zql_lr = metrics.accuracy_score(y_lr_pred,y_test)
print('预测准确率', zql_lr)
print(classification_report(y_test, y_rf_pred))

预测准确率 0.899392369757944
              precision    recall  f1-score   support

    Question       0.97      0.85      0.91      1010
  noQuestion       0.98      1.00      0.99      9029

    accuracy                           0.98     10039
   macro avg       0.98      0.92      0.95     10039
weighted avg       0.98      0.98      0.98     10039



### SVC

In [15]:
#用SVC分类器来拟合训练数据
svc = SVC(C=1, break_ties = False, cache_size = 200, class_weight = None, coef0 = 0.0,
         decision_function_shape='ovr',degree=3,gamma = 20,kernel='rbf',
        probability = True, random_state = None, shrinking = True, tol = 0.001,verbose = False)
svc.fit(X_train, y_train)
y_svc_pred = svc.predict(X_test)

In [16]:
zql_svc = metrics.accuracy_score(y_svc_pred,y_test)
print('预测准确率', zql_svc)
# print(classification_report(y_test, y_svc_pred))

预测准确率 0.9801773084968622


### GBDT

In [17]:
#用随机森林分类器来拟合训练数据
gbdt = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                                learning_rate=0.7, loss='exponential', max_depth=3,
                                max_features='auto', max_leaf_nodes = None,
                                min_impurity_decrease=0.0, 
                                min_samples_leaf=1,min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators =25,
                                n_iter_no_change=None,
                                random_state=30, subsample=1.0, tol=0.0001,
                                validation_fraction=0.1,verbose=0,
                                warm_start=False)
gbdt.fit(X_train, y_train)
y_gbdt_pred = gbdt.predict(X_test)

d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWa

In [18]:
zql_gbdt = metrics.accuracy_score(y_gbdt_pred,y_test)
print('预测准确率', zql_gbdt)
# print(classification_report(y_test, y_gbdt_pred))

预测准确率 0.9360494073114852


In [19]:
weight = [zql_rf, zql_svc, zql_gbdt]
weight/=sum(weight)
weight

array([0.33898014, 0.33812109, 0.32289877])

In [20]:
weight

array([0.33898014, 0.33812109, 0.32289877])

### 软投票集成学习模型

In [21]:
vote2 = VotingClassifier(estimators= [('RF',rf),('SVC',svc),('GBDT',gbdt)], voting = 'soft', weights = weight)
vote2.fit(X_train, y_train)
y_vote2_pred = vote2.predict(X_test)

d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
d:\ANACONDA\lib\site-packages\sklearn\tree\_classes.py:277: FutureWa

In [22]:
zql = metrics.accuracy_score(y_vote2_pred,y_test)
print('预测准确率', zql)


预测准确率 0.9818707042534117


In [23]:
#重新进行标签的修正
for i in range(len(y_vote2_pred)):
    if y_vote2_pred[i] != 'Question':
        if is_question(test_text[i]) == True:
            y_vote2_pred[i] = 'Question'
print(classification_report(y_test, y_vote2_pred))

              precision    recall  f1-score   support

    Question       0.91      0.85      0.88      1010
  noQuestion       0.98      0.99      0.99      9029

    accuracy                           0.98     10039
   macro avg       0.95      0.92      0.93     10039
weighted avg       0.98      0.98      0.98     10039



In [24]:
zql = metrics.accuracy_score(y_vote2_pred,y_test)
print('预测准确率', zql)

预测准确率 0.9764916824384899


In [25]:
import pandas as pd
import numpy as np
post_qa_df = pd.read_excel('post_qa_df.xlsx')
post_qa_df.drop(columns='Unnamed: 0',inplace=True)

In [26]:
for i in range(len(post_qa_df)):
    if type(post_qa_df['Title'][i]) == int:
        post_qa_df = post_qa_df.drop([i],axis=0)

In [27]:
post_qa_df

,Answer_Score,Answer_Body,Answer_CommentCount,Id,CreationDate,Score,Body,CommentCount,AcceptedAnswerId,ViewCount,Title,Tags,AnswerCount,FavoriteCount
0,522,<p>An explicit cast to <code>double</code> lik...,1,4,2008-07-31T21:42:52.667,794,<p>I want to assign the decimal variable &quot...,8,7.0,70633,How to convert Decimal to Double in C#?,<c#><floating-point><type-conversion><double><...,14,0.0
1,160,<p>The parent <code>div</code> needs to have a...,0,6,2008-07-31T22:08:08.620,319,<p>I have an absolutely positioned <code>div</...,0,31.0,23465,Why did the width collapse in the percentage w...,<html><css><internet-explorer-7>,7,0.0
2,2385,<p>An easy to understand and simple solution.<...,8,9,2008-07-31T23:40:59.743,2199,<p>Given a <code>DateTime</code> representing ...,10,1404.0,784860,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,74,0.0
3,1070,"<p>Jeff, <a href=""https://stackoverflow.com/qu...",16,11,2008-07-31T23:55:37.967,1644,<p>Given a specific <code>DateTime</code> valu...,3,1248.0,197314,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,41,0.0
4,216,<p>The most popular (==standard?) way of deter...,7,13,2008-08-01T00:42:38.903,701,<p>Is there a standard way for a web server to...,10,NaN,277780,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,27,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55743,1,<p>In terms of actually getting the applicatio...,2,401816,2008-12-30T23:17:53.830,26,<p>How can I make a Click-once deployed app ru...,1,8695121.0,30606,How can I make a Click-once deployed app run a...,<.net><clickonce><startup>,11,0.0
55744,1,"<p>According to the <a href=""http://www.asp.ne...",0,401820,2008-12-30T23:20:23.560,0,<p>Is it possible not to use web service for A...,0,NaN,2169,ASP.net Ajax: Use Code behind function instead...,<asp.net-ajax>,2,NaN
55745,5,"<p>I'm not much of a Ruby person either, but a...",4,401834,2008-12-30T23:27:46.947,3,"<p>Arising out of <a href=""https://stackoverfl...",0,401848.0,2090,How to elegantly compute the anagram signature...,<ruby><algorithm>,3,0.0
55746,4,<p>You can use OpenSSL for that.</p>\n\n<p>tak...,1,401852,2008-12-30T23:36:54.463,4,<p>I'm looking for a good standard based compo...,0,401861.0,4192,Any delphi component for digital signing docum...,<delphi><digital-signature>,3,0.0


In [28]:
post_qa_df['dect_title_que'] = vote2.predict(vectorizer.transform(post_qa_df['Title']))

In [29]:
post_qa_df

,Answer_Score,Answer_Body,Answer_CommentCount,Id,CreationDate,Score,Body,CommentCount,AcceptedAnswerId,ViewCount,Title,Tags,AnswerCount,FavoriteCount,dect_title_que
0,522,<p>An explicit cast to <code>double</code> lik...,1,4,2008-07-31T21:42:52.667,794,<p>I want to assign the decimal variable &quot...,8,7.0,70633,How to convert Decimal to Double in C#?,<c#><floating-point><type-conversion><double><...,14,0.0,noQuestion
1,160,<p>The parent <code>div</code> needs to have a...,0,6,2008-07-31T22:08:08.620,319,<p>I have an absolutely positioned <code>div</...,0,31.0,23465,Why did the width collapse in the percentage w...,<html><css><internet-explorer-7>,7,0.0,Question
2,2385,<p>An easy to understand and simple solution.<...,8,9,2008-07-31T23:40:59.743,2199,<p>Given a <code>DateTime</code> representing ...,10,1404.0,784860,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,74,0.0,noQuestion
3,1070,"<p>Jeff, <a href=""https://stackoverflow.com/qu...",16,11,2008-07-31T23:55:37.967,1644,<p>Given a specific <code>DateTime</code> valu...,3,1248.0,197314,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,41,0.0,noQuestion
4,216,<p>The most popular (==standard?) way of deter...,7,13,2008-08-01T00:42:38.903,701,<p>Is there a standard way for a web server to...,10,NaN,277780,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,27,0.0,noQuestion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55743,1,<p>In terms of actually getting the applicatio...,2,401816,2008-12-30T23:17:53.830,26,<p>How can I make a Click-once deployed app ru...,1,8695121.0,30606,How can I make a Click-once deployed app run a...,<.net><clickonce><startup>,11,0.0,noQuestion
55744,1,"<p>According to the <a href=""http://www.asp.ne...",0,401820,2008-12-30T23:20:23.560,0,<p>Is it possible not to use web service for A...,0,NaN,2169,ASP.net Ajax: Use Code behind function instead...,<asp.net-ajax>,2,NaN,noQuestion
55745,5,"<p>I'm not much of a Ruby person either, but a...",4,401834,2008-12-30T23:27:46.947,3,"<p>Arising out of <a href=""https://stackoverfl...",0,401848.0,2090,How to elegantly compute the anagram signature...,<ruby><algorithm>,3,0.0,noQuestion
55746,4,<p>You can use OpenSSL for that.</p>\n\n<p>tak...,1,401852,2008-12-30T23:36:54.463,4,<p>I'm looking for a good standard based compo...,0,401861.0,4192,Any delphi component for digital signing docum...,<delphi><digital-signature>,3,0.0,Question


In [31]:
#因为刚刚删除了一个记录，所以要重新重新分配索引
post_qa_df.index = range(len(post_qa_df))
#重新进行标签的修正
for i in range(len(post_qa_df)):
    if post_qa_df['dect_title_que'][i] != 'Question':
        if is_question(post_qa_df['Title'][i]) == True:
            post_qa_df['dect_title_que'][i] = 'Question'

C:\Users\wang\AppData\Local\Temp\ipykernel_3056\1559716716.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_qa_df['dect_title_que'][i] = 'Question'


In [34]:
str_tmp = "ABCD"
str_tmp.lower()

'abcd'

In [ ]:
post_qa_df

,Answer_Score,Answer_Body,Answer_CommentCount,Id,CreationDate,Score,Body,CommentCount,AcceptedAnswerId,ViewCount,Title,Tags,AnswerCount,FavoriteCount,dect_title_que
0,522,<p>An explicit cast to <code>double</code> lik...,1,4,2008-07-31T21:42:52.667,794,<p>I want to assign the decimal variable &quot...,8,7.0,70633,How to convert Decimal to Double in C#?,<c#><floating-point><type-conversion><double><...,14,0.0,Question
1,160,<p>The parent <code>div</code> needs to have a...,0,6,2008-07-31T22:08:08.620,319,<p>I have an absolutely positioned <code>div</...,0,31.0,23465,Why did the width collapse in the percentage w...,<html><css><internet-explorer-7>,7,0.0,Question
2,2385,<p>An easy to understand and simple solution.<...,8,9,2008-07-31T23:40:59.743,2199,<p>Given a <code>DateTime</code> representing ...,10,1404.0,784860,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,74,0.0,Question
3,1070,"<p>Jeff, <a href=""https://stackoverflow.com/qu...",16,11,2008-07-31T23:55:37.967,1644,<p>Given a specific <code>DateTime</code> valu...,3,1248.0,197314,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,41,0.0,noQuestion
4,216,<p>The most popular (==standard?) way of deter...,7,13,2008-08-01T00:42:38.903,701,<p>Is there a standard way for a web server to...,10,NaN,277780,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,27,0.0,noQuestion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55741,1,<p>In terms of actually getting the applicatio...,2,401816,2008-12-30T23:17:53.830,26,<p>How can I make a Click-once deployed app ru...,1,8695121.0,30606,How can I make a Click-once deployed app run a...,<.net><clickonce><startup>,11,0.0,Question
55742,1,"<p>According to the <a href=""http://www.asp.ne...",0,401820,2008-12-30T23:20:23.560,0,<p>Is it possible not to use web service for A...,0,NaN,2169,ASP.net Ajax: Use Code behind function instead...,<asp.net-ajax>,2,NaN,noQuestion
55743,5,"<p>I'm not much of a Ruby person either, but a...",4,401834,2008-12-30T23:27:46.947,3,"<p>Arising out of <a href=""https://stackoverfl...",0,401848.0,2090,How to elegantly compute the anagram signature...,<ruby><algorithm>,3,0.0,Question
55744,4,<p>You can use OpenSSL for that.</p>\n\n<p>tak...,1,401852,2008-12-30T23:36:54.463,4,<p>I'm looking for a good standard based compo...,0,401861.0,4192,Any delphi component for digital signing docum...,<delphi><digital-signature>,3,0.0,Question


In [ ]:
post_qa_question = post_qa_df.copy() 
for i in range(len(post_qa_question)):
    #观察运行进度
    if i%5000 == 0:
        print('现在i的值是',i)
    #如果为noquestion，就将该行删掉
    if post_qa_question['dect_title_que'][i] == 'noQuestion':
        post_qa_question.drop([i],inplace=True)

现在i的值是 0


KeyboardInterrupt: 

In [ ]:
post_qa_question.index = range(len(post_qa_question))

In [ ]:
post_qa_question

,Answer_Score,Answer_Body,Answer_CommentCount,Id,CreationDate,Score,Body,CommentCount,AcceptedAnswerId,ViewCount,Title,Tags,AnswerCount,FavoriteCount,dect_title_que
0,522,<p>An explicit cast to <code>double</code> lik...,1,4,2008-07-31T21:42:52.667,794,<p>I want to assign the decimal variable &quot...,8,7.0,70633,How to convert Decimal to Double in C#?,<c#><floating-point><type-conversion><double><...,14,0.0,Question
1,160,<p>The parent <code>div</code> needs to have a...,0,6,2008-07-31T22:08:08.620,319,<p>I have an absolutely positioned <code>div</...,0,31.0,23465,Why did the width collapse in the percentage w...,<html><css><internet-explorer-7>,7,0.0,Question
2,2385,<p>An easy to understand and simple solution.<...,8,9,2008-07-31T23:40:59.743,2199,<p>Given a <code>DateTime</code> representing ...,10,1404.0,784860,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,74,0.0,Question
3,222,"<p>The <a href=""http://en.wikipedia.org/wiki/M...",2,19,2008-08-01T05:21:22.257,351,<p>I'm looking for the fastest way to obtain t...,17,531.0,67439,What is the fastest way to get the value of π?,<performance><algorithm><language-agnostic><un...,24,0.0,Question
4,45,<p>I've had no trouble using the BSD sockets A...,0,25,2008-08-01T12:13:50.207,175,<p>I'm having issues getting the C sockets API...,1,1443907.0,15845,How to use the C socket API in C++ on z/OS,<c++><c><sockets><mainframe><zos>,9,0.0,Question
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24532,6,"<p>The H gives it away, in this case the libra...",1,401812,2008-12-30T23:16:24.660,4,<p>What is the proper way to cast from an 'OLE...,0,401879.0,4114,What is the proper way to cast from an 'OLE_HA...,<c++><windows><winapi><mfc><casting>,5,NaN,Question
24533,1,<p>In terms of actually getting the applicatio...,2,401816,2008-12-30T23:17:53.830,26,<p>How can I make a Click-once deployed app ru...,1,8695121.0,30606,How can I make a Click-once deployed app run a...,<.net><clickonce><startup>,11,0.0,Question
24534,5,"<p>I'm not much of a Ruby person either, but a...",4,401834,2008-12-30T23:27:46.947,3,"<p>Arising out of <a href=""https://stackoverfl...",0,401848.0,2090,How to elegantly compute the anagram signature...,<ruby><algorithm>,3,0.0,Question
24535,4,<p>You can use OpenSSL for that.</p>\n\n<p>tak...,1,401852,2008-12-30T23:36:54.463,4,<p>I'm looking for a good standard based compo...,0,401861.0,4192,Any delphi component for digital signing docum...,<delphi><digital-signature>,3,0.0,Question


In [ ]:
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',150)
pd.set_option('display.max_columns', None)  # 设置显示最大行

In [ ]:
#去掉答案html格式
import re
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
for i in range(len(post_qa_question)):
    post_qa_question['Answer_Body'][i] = remove_html_tags(post_qa_question['Answer_Body'][i])
    post_qa_question['Answer_Body'][i] = post_qa_question['Answer_Body'][i].lower()


C:\Users\wang\AppData\Local\Temp\ipykernel_26220\3940936210.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_qa_question['Answer_Body'][i] = remove_html_tags(post_qa_question['Answer_Body'][i])
C:\Users\wang\AppData\Local\Temp\ipykernel_26220\3940936210.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_qa_question['Answer_Body'][i] = post_qa_question['Answer_Body'][i].lower()


In [ ]:
#将post_qa_question写入csv和excel
post_qa_question.to_csv('post_qa_question.csv')
post_qa_question.to_excel('post_qa_question.xlsx')